In [1]:
!pip install astronn equinox einops

In [2]:
from astroNN.datasets import load_galaxy10
from astroNN.datasets.galaxy10 import galaxy10cls_lookup, galaxy10_confusion
import numpy as np
import matplotlib.pyplot as plt
import jax

# Galaxy 10 DECals

In [ ]:
# To load images and labels (will download automatically at the first time)
# First time downloading location will be ~/.astroNN/datasets/
images, labels = load_galaxy10()

/root/.astroNN/datasets/Galaxy10_DECals.h5 was found!


In [ ]:
fig, axs = plt.subplots(2, 5, figsize=(12,5), layout='constrained')

for ax, label in zip(np.ravel(axs), range(10)):
    idx = np.where(labels == label)[0][0]
    image = images[idx]

    ax.imshow(image)
    ax.axis('off')
    ax.set_title(galaxy10cls_lookup(label))

#### Remove classes that are not Spiral-like

In [ ]:
remove_labels = range(5)

for label in remove_labels:
    idx = labels != label
    labels = labels[idx]
    images = images[idx]

#### Shift labels to 0-4

In [ ]:
labels = labels - 5

galaxy10cls_lookup_shift = lambda x: galaxy10cls_lookup(x+5)

#### Rescale images

In [ ]:
images.shape

In [ ]:
images = jax.vmap(lambda x: x / 255)(images)

# crop the image a bit to get 64x64 shape
def crop_center(images, cropx, cropy):
    _, y, x, _ = images.shape
    startx = x // 2 -(cropx // 2)
    starty = y // 2 -(cropy // 2)
    return images[:, starty:starty+cropy, startx:startx+cropx, :]

images = crop_center(images=images, cropx=64, cropy=64)

In [ ]:
images.shape

In [ ]:
plt.imshow(images[0])

#### Shuffle images

In [ ]:
rnd_idx = np.random.choice(range(len(images)), replace=False, size=len(images))
images = images[rnd_idx]
labels = labels[rnd_idx]

In [ ]:
fig, axs = plt.subplots(2, 5, figsize=(12,5), layout='constrained')

for ax, idx in zip(np.ravel(axs), range(10)):
    image = images[idx]
    label = labels[idx]

    ax.imshow(image)
    ax.axis('off')
    ax.set_title(galaxy10cls_lookup_shift(label))

In [ ]:
confusion_matrix = np.zeros((5,5))


# create the confusion matrix
for counter, i in enumerate(labels):
    confusion_matrix[i, labels[counter]] += 1

# Plot the confusion matrix
# galaxy10_confusion(confusion_matrix)

In [ ]:
galaxy10_confusion(confusion_matrix)

# Task: Find the galaxy that looks most similar to Milky Way

<img src="https://www.nasa.gov/images/content/188404main_hurt_Milky_Way_2005-590_lg.jpg" width=300>

## Bonus task: Genereate new images that look like the Milky Way